In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import html
import scrapy
from time import sleep
import urllib3
import json
from selenium import webdriver
import random

In [2]:
def parser(link):

    encabezados = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
    }

    resp = requests.get(link, headers=encabezados, verify=False)
    resp = resp.text

    #soup = get_Soup('https://minciencias.gov.co/convocatorias/todas')
    parser = html.fromstring(resp)
    urllib3.disable_warnings()

    return parser

In [3]:
pagina = 'https://dst.gov.in/archive-call-for-proposals'



parser_india = parser(pagina)

ult_page = int(parser_india.xpath('//div[@class="item-list"]/ul//li[@class="pager-last last"]/a/@href')[0].split('page=')[1])+1

india_DCYT_final = pd.DataFrame()

base = 'https://dst.gov.in/archive-call-for-proposals?page='

for n_page in range(1, ult_page+1):

    page = base + str(n_page)
    india_DCYT = pd.DataFrame()
    
    parser_india = parser(page) 

    #Titulo
    titulo = parser_india.xpath('//table/tbody//tr//td[@class="views-field views-field-title"]/a/text()')

    #PDF
    pdfs = []
    for i in range(1, len(titulo)+1):
        

        pdf = parser_india.xpath('//table/tbody/tr[' + str(i) + ']/td[2][@class="views-field views-field-field-attached"]//@href')

        if len(pdf) == 0:
            pdf = ''
            pdfs.append(pdf)
        else:
            pdfs.append(pdf[0])

    # Fecha de apertura
    fecha_apertura = parser_india.xpath('//table/tbody//tr//td[3]/span[@class="date-display-single"]//text()')
    
    #Fecha cierre
    fecha_cierre = parser_india.xpath('//table/tbody//tr//td[4]/span[@class="date-display-single"]//text()')

    india_DCYT['Título'] = titulo
    india_DCYT['PDFs'] = pdfs
    india_DCYT['Fecha Apertura'] = fecha_apertura
    india_DCYT['Fecha Cierre'] = fecha_cierre

    india_DCYT_final = pd.concat([india_DCYT_final, india_DCYT])




/home/duban/anaconda3/envs/scraping-clacso/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dst.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [6]:
india_DCYT_final = india_DCYT_final.reset_index(drop=True)

india_DCYT_final.to_excel('India_departamento de ciencia y tecnología.xlsx')

In [7]:
india_DCYT_final

,Título,PDFs,Fecha Apertura,Fecha Cierre
0,2nd Call for Proposal for DST-STI Policy Fello...,https://dst.gov.in/sites/default/files/DST_STI...,03/05/2017,30/05/2017
1,DST-DAAD Germany Joint Call For Proposal 2017,https://dst.gov.in/sites/default/files/DST-DAA...,19/04/2017,14/06/2017
2,Call for applications under the second phase o...,https://dst.gov.in/sites/default/files/Final%2...,18/04/2017,16/06/2017
3,Advertisement for submission of research propo...,https://dst.gov.in/sites/default/files/CSRI-AD...,12/05/2017,30/06/2017
4,Invites Science and Technology Project Proposa...,https://dst.gov.in/sites/default/files/announc...,29/06/2017,15/07/2017
...,...,...,...,...
172,Extension of Last Date for proposal submission...,https://dst.gov.in/sites/default/files/CSRI-Ad...,15/02/2021,15/03/2021
173,ACT Call 3 Stage 2: Call for shortlisted prepr...,https://dst.gov.in/sites/default/files/ACT%20C...,08/01/2021,15/03/2021
174,Grand Challenge for BDTD Programme of TDT Divi...,,07/09/2020,31/03/2021
175,Indo-Italian call for proposals- date extended...,https://dst.gov.in/sites/default/files/Indo-It...,15/02/2021,30/04/2021
